# BGP AS-Level Topology: Graph Feature Extraction Pipeline**Purpose:** Download RIPE RIS MRT data (RIB dumps), parse with bgpkit-parser,construct AS-level topology graphs, and extract comprehensive graph-theoretic features for BGP anomaly detection.**Features extracted:**- **16 Graph-level metrics:** assortativity, diameter, algebraic connectivity, spectral radius, symmetry ratio, natural connectivity, effective graph resistance, spanning tree count, weighted spectrum stats, percolation limit, node/edge connectivity, clustering coefficient, density, rich-club coefficient, betweenness distribution stats, k-core decomposition metrics- **10 Node-level metrics:** degree centrality, betweenness centrality, closeness centrality, eigenvector centrality, PageRank, local clustering coefficient, average neighbor degree, node clique number, eccentricity, k-shell/core number**Enhancements over original:**- Private/reserved ASN filtering (RFC 6996, RFC 7300)- Edge weight tracking (number of AS paths traversing each link)- Consolidated matrix computations (adjacency/Laplacian computed once, reused)- Fixed symmetry ratio and natural connectivity for partial spectrum- Eliminated duplicate betweenness and k-core computations- Improved eccentricity handling for large graphs via NetworKit BFS- Proper clique number fallback with greedy approximation**Data source:** Configurable RIPE RIS collector and time period. RIB-only mode.**References:**- Willinger, W. & Roughan, M. "Internet Topology Research Redux," *Recent Advances in Networking*, ACM SIGCOMM (2013)- Newman, M.E.J. *Networks: An Introduction*, Oxford University Press (2010)- Li, L. et al. "A First-Principles Approach to Understanding the Internet's Router-Level Topology," *ACM SIGCOMM* (2004)- Sanchez, O.R. et al. "Comparing ML Algorithms for BGP Anomaly Detection using Graph Features," *Big-DAMA '19* (2019)---

## 1. Installation & Imports

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install pybgpkit networkx scipy numpy pandas matplotlib seaborn tqdm

In [ ]:
import os
import time
import json
import logging
import warnings
from datetime import datetime, timedelta, timezone
from pathlib import Path
from collections import defaultdict
from typing import Optional, Dict, List, Tuple, Set

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
from scipy import sparse
from scipy.sparse.linalg import eigsh
from scipy import stats as sp_stats
import bgpkit

warnings.filterwarnings('ignore', category=FutureWarning)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Optional: NetworKit for high-performance computation on large graphs
try:
    import networkit as nk
    HAS_NETWORKIT = True
    logger.info("NetworKit available - will use for performance-critical computations")
except ImportError:
    HAS_NETWORKIT = False
    logger.info("NetworKit not available - using NetworkX only (slower for large graphs)")

print(f"NetworkX version: {nx.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"NetworKit available: {HAS_NETWORKIT}")

## 2. Configuration

All parameters are configurable. Adjust the collector, date range, and processing mode below.

In [ ]:
# ============================================================================# CONFIGURATION - Modify these parameters as needed# ============================================================================# --- RIPE RIS Collector ---COLLECTOR = "rrc04"# --- Date Range ---START_DATE = "2025-11-17"  # YYYY-MM-DDEND_DATE = "2025-11-18"    # YYYY-MM-DD (inclusive)# --- Output Directories ---BASE_DIR = Path("./bgp_graph_features")DATA_DIR = BASE_DIR / "data"OUTPUT_DIR = BASE_DIR / "output"FIGURES_DIR = BASE_DIR / "figures"for d in [DATA_DIR, OUTPUT_DIR, FIGURES_DIR]:    d.mkdir(parents=True, exist_ok=True)# --- Private / Reserved ASN Ranges (to be filtered out) ---# RFC 6996: 64512-65534 (16-bit private), 4200000000-4294967294 (32-bit private)# RFC 7300: 65535 (last 16-bit), 4294967295 (last 32-bit)# IANA reserved: 0, 23456 (AS_TRANS, RFC 6793)PRIVATE_ASNS = (    set(range(64512, 65535))              # 16-bit private    | set(range(4200000000, 4294967295))  # 32-bit private    | {0, 23456, 65535, 4294967295}       # reserved / special)def is_valid_public_asn(asn: int) -> bool:    """Return True if the ASN is a valid public (non-private, non-reserved) ASN."""    return asn not in PRIVATE_ASNS# --- Performance Settings ---BETWEENNESS_SAMPLE_K = 500COMPUTE_SPECTRAL = TrueMAX_NODES_FOR_CLIQUE = 5000# --- RIPE RIS URL Pattern ---RIPE_BASE_URL = "https://data.ris.ripe.net"print(f"Configuration:")print(f"  Collector: {COLLECTOR}")print(f"  Date range: {START_DATE} to {END_DATE}")print(f"  Output: {OUTPUT_DIR}")print(f"  Private ASN filter: {len(PRIVATE_ASNS):,} ASNs will be excluded")

## 3. Data Discovery & DownloadUses BGPKIT Broker to discover available MRT files for the configured collector and time range,then constructs direct URLs for RIB dumps (`bview.*`).**RIPE RIS URL pattern:**```https://data.ris.ripe.net/{collector}/{YYYY.MM}/bview.{YYYYMMDD}.{HHMM}.gz```RIB dumps are generated every **8 hours** at 00:00, 08:00, 16:00 UTC.

In [ ]:
def generate_rib_urls(collector: str, start_date: str, end_date: str) -> List[str]:    """    Generate URLs for all RIB dump files in the given date range.    RIB dumps are available at 00:00, 08:00, 16:00 UTC daily.    """    urls = []    start = datetime.strptime(start_date, "%Y-%m-%d")    end = datetime.strptime(end_date, "%Y-%m-%d")    rib_hours = [0, 8, 16]    current = start    while current <= end:        year_month = current.strftime("%Y.%m")        for hour in rib_hours:            ts = current.replace(hour=hour, minute=0)            if ts < start or ts > end + timedelta(days=1) - timedelta(seconds=1):                continue            filename = f"bview.{ts.strftime('%Y%m%d.%H%M')}.gz"            url = f"{RIPE_BASE_URL}/{collector}/{year_month}/{filename}"            urls.append(url)        current += timedelta(days=1)    return urlsdef discover_files_via_broker(collector: str, start_date: str, end_date: str) -> List[dict]:    """    Use BGPKIT Broker to discover available RIB MRT files.    Falls back to URL generation if Broker is unavailable.    """    try:        broker = bgpkit.Broker()        items = broker.query(            ts_start=f"{start_date}T00:00:00",            ts_end=f"{end_date}T23:59:59",            collector_id=collector,            data_type="rib"        )        if items:            logger.info(f"Broker found {len(items)} RIB files")            return items    except Exception as e:        logger.warning(f"Broker query failed: {e}. Falling back to URL generation.")    urls = generate_rib_urls(collector, start_date, end_date)    logger.info(f"Generated {len(urls)} RIB URLs")    return [{"url": url} for url in urls]# Discover RIB filesrib_files = discover_files_via_broker(COLLECTOR, START_DATE, END_DATE)print(f"\nRIB files discovered: {len(rib_files)}")for f in rib_files[:5]:    url = f['url'] if isinstance(f, dict) else f.url    print(f"  {url}")

## 4. MRT Parsing with bgpkit-parser

**bgpkit-parser** is a Rust-based MRT parser with Python bindings (`pybgpkit`).  
It handles gzip decompression and MRT parsing transparently.

### Workflow (CSV-first approach)
1. **Download** MRT `.gz` files locally (saved to `data/mrt_files/`)
2. **Parse** each file into structured rows matching the standard TABLE_DUMP2 format
3. **Save** all rows as a single CSV (saved to `output/`)
4. **Build** AS edges from the CSV for graph construction

### CSV Schema (TABLE_DUMP2 format for RIB entries)

| Column | Description | Example |
|--------|-------------|---------|
| `MRT_Type` | Always `TABLE_DUMP2` for RIB dumps | `TABLE_DUMP2` |
| `Timestamp` | Dump timestamp (UTC) | `2025-11-17 00:00:00` |
| `Entry_Type` | Always `B` (table entry) for RIB | `B` |
| `Peer_IP` | IP of the BGP peer | `198.32.132.97` |
| `Peer_AS` | ASN of the peer | `13335` |
| `Prefix` | Announced prefix | `1.0.0.0/24` |
| `AS_Path` | Full AS path (space-separated) | `3356 1299 13335` |
| `Origin` | ORIGIN attribute (IGP/EGP/INCOMPLETE) | `IGP` |
| `Next_Hop` | Next-hop IP | `198.32.132.97` |
| `Local_Pref` | LOCAL_PREF value | `100` |
| `MED` | Multi-Exit Discriminator | `0` |
| `Community` | BGP communities (space-separated) | `3356:100 3356:123` |
| `Atomic_Aggregate` | Atomic aggregate flag | `AG` or empty |
| `Aggregator` | Aggregator AS and IP | `13335 198.32.132.97` |

**Note:** RIB dumps use `TABLE_DUMP2` with entry type `B`, unlike UPDATE files which use `BGP4MP` with `A` (announcement) and `W` (withdrawal).

### Graph construction from AS_PATH
1. Parse AS_PATH string (space-separated ASNs)
2. Remove AS prepending (consecutive duplicates)
3. Skip AS_SET entries (e.g., `{1234,5678}`)
4. Filter private/reserved ASNs (RFC 6996, RFC 7300)
5. Extract pairwise adjacent links -> undirected edges

In [ ]:
import urllib.request

def parse_as_path(as_path_str: str) -> List[int]:
    """
    Parse an AS_PATH string into a deduplicated list of valid public ASNs.
    Handles:
    - Standard AS paths: "3356 1299 13335"
    - AS prepending: "3356 3356 3356 1299" -> [3356, 1299]
    - AS_SETs: "{1234,5678}" -> skipped entirely
    - Private/reserved ASNs: filtered out (RFC 6996, RFC 7300)
    """
    if not as_path_str:
        return []

    tokens = as_path_str.split()
    deduped = []
    for token in tokens:
        if '{' in token or '}' in token:
            continue
        try:
            asn = int(token)
            if not is_valid_public_asn(asn):
                continue
            if not deduped or asn != deduped[-1]:
                deduped.append(asn)
        except ValueError:
            continue
    return deduped


def extract_edges_from_as_path(as_path: List[int]) -> List[Tuple[int, int]]:
    """
    Extract pairwise AS adjacency edges from a parsed AS path.
    Returns a list (not set) so callers can count occurrences for weighting.
    """
    edges = []
    for i in range(len(as_path) - 1):
        edge = tuple(sorted([as_path[i], as_path[i + 1]]))
        if edge[0] != edge[1]:
            edges.append(edge)
    return edges


def download_mrt_file(url: str, dest_dir: Path) -> Path:
    """Download an MRT file and save it locally. Returns the local file path."""
    filename = url.split('/')[-1]
    local_path = dest_dir / filename

    if local_path.exists():
        size_mb = local_path.stat().st_size / (1024 * 1024)
        logger.info(f"  Already downloaded: {filename} ({size_mb:.1f} MB)")
        return local_path

    logger.info(f"  Downloading: {filename}")
    t0 = time.time()
    urllib.request.urlretrieve(url, str(local_path))
    elapsed = time.time() - t0
    size_mb = local_path.stat().st_size / (1024 * 1024)
    logger.info(f"  Saved: {filename} ({size_mb:.1f} MB, {elapsed:.1f}s)")
    return local_path


def parse_mrt_to_rows(file_path: str) -> Tuple[List[dict], dict]:
    """
    Parse a single MRT RIB dump file into structured TABLE_DUMP2 rows.

    Each RIB entry is mapped to:
        TABLE_DUMP2|timestamp|B|peer_ip|peer_as|prefix|as_path|origin|
        next_hop|local_pref|med|community|atomic_agg|aggregator

    Returns:
        rows: list of dicts (one per RIB entry)
        stats: parsing statistics
    """
    rows = []
    stats = {
        'total_elements': 0,
        'announcements': 0,
        'withdrawals': 0,
        'unique_prefixes': set(),
        'unique_peers': set(),
        'parse_errors': 0,
    }

    logger.info(f"  Parsing: {Path(file_path).name}")
    t0 = time.time()

    try:
        parser = bgpkit.Parser(url=str(file_path))
        for elem in parser:
            stats['total_elements'] += 1

            if elem.elem_type == "W":
                stats['withdrawals'] += 1
                continue

            stats['announcements'] += 1

            if elem.prefix:
                stats['unique_prefixes'].add(elem.prefix)
            if elem.peer_asn:
                stats['unique_peers'].add(elem.peer_asn)

            # Convert timestamp to readable UTC string
            ts = datetime.fromtimestamp(
                elem.timestamp, tz=timezone.utc
            ).strftime('%Y-%m-%d %H:%M:%S')

            # Build community string (space-separated)
            communities = ''
            if elem.communities:
                communities = ' '.join(str(c) for c in elem.communities)

            # Build aggregator string
            aggregator = ''
            aggr_asn = getattr(elem, 'aggr_asn', None)
            aggr_ip = getattr(elem, 'aggr_ip', None)
            if aggr_asn:
                aggregator = f"{aggr_asn} {aggr_ip}".strip() if aggr_ip else str(aggr_asn)

            row = {
                'MRT_Type': 'TABLE_DUMP2',
                'Timestamp': ts,
                'Entry_Type': 'B',
                'Peer_IP': elem.peer_ip or '',
                'Peer_AS': elem.peer_asn if elem.peer_asn else '',
                'Prefix': elem.prefix or '',
                'AS_Path': elem.as_path or '',
                'Origin': elem.origin or '',
                'Next_Hop': elem.next_hop or '',
                'Local_Pref': elem.local_pref if elem.local_pref is not None else '',
                'MED': elem.med if elem.med is not None else '',
                'Community': communities,
                'Atomic_Aggregate': 'AG' if elem.atomic else '',
                'Aggregator': aggregator,
            }
            rows.append(row)

    except Exception as e:
        logger.error(f"Error parsing {file_path}: {e}")
        stats['parse_errors'] += 1

    elapsed = time.time() - t0
    stats['unique_prefixes'] = len(stats['unique_prefixes'])
    stats['unique_peers'] = len(stats['unique_peers'])
    stats['rows_parsed'] = len(rows)
    stats['parse_time_sec'] = round(elapsed, 2)

    logger.info(
        f"  -> {stats['total_elements']:,} elements, "
        f"{len(rows):,} rows, {stats['unique_prefixes']:,} prefixes, "
        f"{stats['unique_peers']:,} peers in {elapsed:.1f}s"
    )
    return rows, stats


print("Parse functions defined:")
print("  - parse_as_path(as_path_str) -> List[int]")
print("  - extract_edges_from_as_path(as_path) -> List[Tuple[int,int]]")
print("  - download_mrt_file(url, dest_dir) -> Path")
print("  - parse_mrt_to_rows(file_path) -> (rows, stats)")

In [ ]:
# ============================================================================
# STEP 1: Download MRT files locally & parse to structured CSV
# ============================================================================
# This cell downloads the raw MRT .gz files and parses every RIB entry
# into a CSV with TABLE_DUMP2 format. Both artifacts are saved to disk.
# Re-running this cell will skip already-downloaded files.
# ============================================================================

MRT_DIR = DATA_DIR / "mrt_files"
MRT_DIR.mkdir(parents=True, exist_ok=True)

all_rows = []
all_stats = []

print("=" * 70)
print("STEP 1: DOWNLOAD MRT FILES & PARSE TO CSV")
print("=" * 70)

for i, f in enumerate(rib_files):
    url = f['url'] if isinstance(f, dict) else f.url
    print(f"\n[{i+1}/{len(rib_files)}] {url}")

    # --- Download MRT file locally ---
    local_path = download_mrt_file(url, MRT_DIR)

    # --- Parse to structured rows ---
    rows, stats = parse_mrt_to_rows(str(local_path))
    stats['file_type'] = 'rib'
    stats['url'] = url
    stats['local_path'] = str(local_path)
    all_rows.extend(rows)
    all_stats.append(stats)
    print(f"  Running total: {len(all_rows):,} rows")

# --- Save combined CSV ---
csv_filename = f"rib_parsed_{COLLECTOR}_{START_DATE}_{END_DATE}.csv"
csv_path = OUTPUT_DIR / csv_filename
print(f"\nSaving parsed CSV to {csv_path}...")
rib_df = pd.DataFrame(all_rows)
rib_df.to_csv(csv_path, index=False)
csv_size_mb = csv_path.stat().st_size / (1024 * 1024)

# --- Save parsing stats ---
stats_df = pd.DataFrame(all_stats)
stats_df.to_csv(OUTPUT_DIR / "parsing_stats.csv", index=False)

print(f"\n{'=' * 70}")
print(f"DOWNLOAD & PARSE COMPLETE")
print(f"{'=' * 70}")
print(f"  MRT files saved to: {MRT_DIR}/")
for mrt_file in sorted(MRT_DIR.iterdir()):
    sz = mrt_file.stat().st_size / (1024 * 1024)
    print(f"    {mrt_file.name:<40} {sz:.1f} MB")
print(f"  CSV saved to: {csv_path} ({csv_size_mb:.1f} MB)")
print(f"  Total rows: {len(rib_df):,}")
print(f"  Columns: {list(rib_df.columns)}")
print(f"  Files processed: {len(all_stats)}")
print(f"{'=' * 70}")

# Preview first rows
rib_df.head()

In [ ]:
# ============================================================================
# STEP 2: Load CSV & build AS topology edges
# ============================================================================
# This cell can run independently — if rib_df is not in memory, it loads
# the CSV from disk. This means you only parse MRT files once.
# ============================================================================
from collections import Counter

csv_path = OUTPUT_DIR / f"rib_parsed_{COLLECTOR}_{START_DATE}_{END_DATE}.csv"

if 'rib_df' not in dir() or rib_df is None:
    print(f"Loading CSV from {csv_path}...")
    rib_df = pd.read_csv(csv_path)

print(f"Building AS topology edges from {len(rib_df):,} RIB entries...")
print("=" * 70)

# --- Check for private/reserved ASNs in raw data ---
private_found = set()
all_edges_list = []

for as_path_raw in rib_df['AS_Path'].dropna().astype(str):
    if as_path_raw == '' or as_path_raw == 'nan':
        continue

    # Check for private ASNs (before filtering)
    for token in as_path_raw.split():
        if '{' in token or '}' in token:
            continue
        try:
            asn = int(token)
            if asn in PRIVATE_ASNS:
                private_found.add(asn)
        except ValueError:
            pass

    # Parse AS path (filters private ASNs + removes prepending)
    as_path = parse_as_path(as_path_raw)
    edges = extract_edges_from_as_path(as_path)
    all_edges_list.extend(edges)

# --- Report private ASN findings ---
print(f"\nPrivate/reserved ASN check:")
if private_found:
    print(f"  Found {len(private_found)} private/reserved ASNs in raw data:")
    for asn in sorted(private_found):
        label = "AS_TRANS" if asn == 23456 else "private" if asn >= 64512 else "reserved"
        print(f"    AS{asn} ({label})")
    print(f"  These have been filtered out of the edge set.")
else:
    print(f"  None found — all ASNs in the data are public. Filter had no effect.")

# --- Build edge weights from observation counts ---
edge_counts = Counter(all_edges_list)
all_edges = set(edge_counts.keys())

# --- Summary ---
print(f"\n{'=' * 70}")
print(f"EDGE EXTRACTION COMPLETE")
print(f"  Total unique edges: {len(all_edges):,}")
print(f"  Total edge observations: {len(all_edges_list):,}")
if edge_counts:
    weights = sorted(edge_counts.values())
    print(f"  Weight range: min={weights[0]:,}, "
          f"max={weights[-1]:,}, "
          f"median={weights[len(weights)//2]:,}")
print(f"{'=' * 70}")

## 5. AS-Level Graph Construction

Build an undirected NetworkX graph from the extracted AS adjacency pairs.
Each node represents an Autonomous System (ASN) and each edge represents
an observed routing adjacency in BGP AS_PATH attributes.

In [ ]:
def build_as_graph(edges: Set[Tuple[int, int]],                   edge_weights: dict = None) -> nx.Graph:    """    Construct an undirected AS-level topology graph from adjacency pairs.    Optionally attach edge weights (number of AS-path observations).    """    G = nx.Graph()    if edge_weights:        for (u, v), w in edge_weights.items():            G.add_edge(u, v, weight=w)    else:        G.add_edges_from(edges)    G.graph['name'] = f"AS-level topology ({COLLECTOR})"    G.graph['collector'] = COLLECTOR    G.graph['start_date'] = START_DATE    G.graph['end_date'] = END_DATE    G.graph['created'] = datetime.now(timezone.utc).isoformat()    return G# Build the graph with edge weightsG = build_as_graph(all_edges, edge_weights=edge_counts)print(f"AS-Level Topology Graph:")print(f"  Nodes (ASes): {G.number_of_nodes():,}")print(f"  Edges (links): {G.number_of_edges():,}")print(f"  Connected: {nx.is_connected(G)}")if not nx.is_connected(G):    components = list(nx.connected_components(G))    sizes = sorted([len(c) for c in components], reverse=True)    print(f"  Connected components: {len(components)}")    print(f"  Largest component: {sizes[0]:,} nodes ({100*sizes[0]/G.number_of_nodes():.1f}%)")    if len(sizes) > 1:        print(f"  2nd largest: {sizes[1]:,} nodes")    print(f"  Components with 1 node: {sizes.count(1)}")    largest_cc = max(components, key=len)    G_lcc = G.subgraph(largest_cc).copy()    print(f"\n  -> Using largest connected component for analysis")else:    G_lcc = Gprint(f"\nAnalysis graph: {G_lcc.number_of_nodes():,} nodes, {G_lcc.number_of_edges():,} edges")

In [ ]:
# Basic degree distribution overview
degrees = [d for _, d in G_lcc.degree()]
degree_series = pd.Series(degrees)

print("Degree distribution statistics:")
print(f"  Min: {degree_series.min()}")
print(f"  Max: {degree_series.max():,}")
print(f"  Mean: {degree_series.mean():.2f}")
print(f"  Median: {degree_series.median():.1f}")
print(f"  Std: {degree_series.std():.2f}")
print(f"  Skewness: {degree_series.skew():.2f}")

# Quick degree distribution plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear histogram
axes[0].hist(degrees, bins=100, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Degree')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Degree Distribution (Linear Scale)')
axes[0].axvline(x=degree_series.mean(), color='red', linestyle='--', label=f'Mean={degree_series.mean():.1f}')
axes[0].legend()

# Log-log CCDF
sorted_deg = np.sort(degrees)[::-1]
ccdf = np.arange(1, len(sorted_deg) + 1) / len(sorted_deg)
axes[1].loglog(sorted_deg, ccdf, '.', markersize=3, color='steelblue')
axes[1].set_xlabel('Degree k')
axes[1].set_ylabel('P(X ≥ k)')
axes[1].set_title('Degree CCDF (Log-Log Scale)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'degree_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'degree_distribution.png'}")

## 6. Graph-Level Feature Extraction

Extract 16 graph-level metrics with proper mathematical definitions and academic citations.

Each metric includes:
- **Definition**: Mathematical formula
- **Interpretation**: What it captures in AS topology
- **Citation**: Authoritative academic reference

In [ ]:
# ============================================================================
# Helper: convert NetworkX → NetworKit (if available)
# ============================================================================

def nx_to_nk(G_nx: nx.Graph):
    """
    Convert a NetworkX graph to NetworKit format.
    Returns (nk_graph, node_mapping) where node_mapping maps NX→NK node IDs.
    """
    if not HAS_NETWORKIT:
        return None, None
    
    # NetworKit requires contiguous integer node IDs starting from 0
    node_list = sorted(G_nx.nodes())
    nx_to_nk_map = {n: i for i, n in enumerate(node_list)}
    nk_to_nx_map = {i: n for n, i in nx_to_nk_map.items()}
    
    G_nk = nk.Graph(len(node_list), weighted=False, directed=False)
    for u, v in G_nx.edges():
        G_nk.addEdge(nx_to_nk_map[u], nx_to_nk_map[v])
    
    return G_nk, nx_to_nk_map, nk_to_nx_map


# Pre-convert if NetworKit is available
if HAS_NETWORKIT:
    G_nk, nx2nk_map, nk2nx_map = nx_to_nk(G_lcc)
    print(f"NetworKit graph: {G_nk.numberOfNodes()} nodes, {G_nk.numberOfEdges()} edges")
else:
    G_nk, nx2nk_map, nk2nx_map = None, None, None
    print("Using NetworkX only")

In [ ]:
# ============================================================================
# Graph-Level Feature Extraction
# ============================================================================

graph_features = {}
n_nodes = G_lcc.number_of_nodes()
n_edges = G_lcc.number_of_edges()

graph_features['n_nodes'] = n_nodes
graph_features['n_edges'] = n_edges

print(f"Extracting graph-level features for {n_nodes:,} nodes, {n_edges:,} edges...")
print("=" * 70)

In [ ]:
# --------------------------------------------------------------------------
# 1. ASSORTATIVITY (Degree Assortativity Coefficient)
# --------------------------------------------------------------------------
# Definition: Pearson correlation of degrees at either end of each edge.
#   r = [M^-1 Σ_i j_i k_i - (M^-1 Σ_i ½(j_i + k_i))^2] /
#       [M^-1 Σ_i ½(j_i² + k_i²) - (M^-1 Σ_i ½(j_i + k_i))^2]
# Range: [-1, 1]. Internet AS graphs are typically disassortative (r < 0).
# Citation: Newman, M.E.J. "Assortative Mixing in Networks,"
#           Physical Review Letters 89, 208701 (2002).
# --------------------------------------------------------------------------

t0 = time.time()
graph_features['assortativity'] = nx.degree_assortativity_coefficient(G_lcc)
print(f"[1/16] Assortativity: {graph_features['assortativity']:.6f}  ({time.time()-t0:.1f}s)")
print(f"        → {'Disassortative' if graph_features['assortativity'] < 0 else 'Assortative'} "
      f"(high-degree nodes preferentially connect to {'low' if graph_features['assortativity'] < 0 else 'high'}-degree nodes)")

In [ ]:
# --------------------------------------------------------------------------
# 2. DENSITY
# --------------------------------------------------------------------------
# Definition: ρ(G) = 2|E| / [|V|(|V|-1)]
# Internet AS graphs are extremely sparse (~10^-5).
# Citation: Standard graph theory definition.
# --------------------------------------------------------------------------

graph_features['density'] = nx.density(G_lcc)
print(f"[2/16] Density: {graph_features['density']:.8f}")

In [ ]:
# --------------------------------------------------------------------------
# 3. CLUSTERING COEFFICIENT (Global & Average Local)
# --------------------------------------------------------------------------
# Global (transitivity): C_global = 3 × triangles / connected_triples
# Average local: C̄ = (1/n) Σ_v C(v), where
#   C(v) = 2|{edges among N(v)}| / [d_v(d_v - 1)]
# Higher than random graphs → regional peering communities, IXP cliques.
# Citation: Watts, D.J. & Strogatz, S.H. "Collective dynamics of 'small-world'
#           networks," Nature 393, 440-442 (1998).
#           Newman, M.E.J. "The structure and function of complex networks,"
#           SIAM Review 45(2), 167-256 (2003).
# --------------------------------------------------------------------------

t0 = time.time()
if HAS_NETWORKIT:
    graph_features['clustering_global'] = nk.globals.ClusteringCoefficient.exactGlobal(G_nk)
    graph_features['clustering_avg_local'] = nk.globals.ClusteringCoefficient.sequentialAvgLocal(G_nk)
else:
    graph_features['clustering_global'] = nx.transitivity(G_lcc)
    graph_features['clustering_avg_local'] = nx.average_clustering(G_lcc)

print(f"[3/16] Clustering coefficient  ({time.time()-t0:.1f}s)")
print(f"        Global (transitivity): {graph_features['clustering_global']:.6f}")
print(f"        Average local: {graph_features['clustering_avg_local']:.6f}")

In [ ]:
# --------------------------------------------------------------------------
# 4. DIAMETER & AVERAGE PATH LENGTH
# --------------------------------------------------------------------------
# Diameter: D = max_{u,v} d(u,v) (longest shortest path)
# Average path length: ℓ = 2/[n(n-1)] · Σ_{u<v} d(u,v)
# Internet AS graph: D ≈ 8-11, ℓ ≈ 3-4 hops (small-world).
# Citation: Watts, D.J. & Strogatz, S.H. Nature 393 (1998).
# --------------------------------------------------------------------------

t0 = time.time()

if HAS_NETWORKIT:
    # NetworKit's iFUB algorithm is near-linear for real-world networks
    diam_algo = nk.distance.Diameter(G_nk, algo=nk.distance.DiameterAlgo.AUTOMATIC)
    diam_algo.run()
    graph_features['diameter'] = diam_algo.getDiameter()[0]
    print(f"[4/16] Diameter: {graph_features['diameter']}  ({time.time()-t0:.1f}s, NetworKit iFUB)")
else:
    # NetworkX: only feasible for smaller graphs
    if n_nodes < 50000:
        graph_features['diameter'] = nx.diameter(G_lcc)
        print(f"[4/16] Diameter: {graph_features['diameter']}  ({time.time()-t0:.1f}s)")
    else:
        # Approximate via BFS from random nodes
        sample_nodes = np.random.choice(list(G_lcc.nodes()), size=min(100, n_nodes), replace=False)
        max_ecc = 0
        for node in sample_nodes:
            ecc = nx.eccentricity(G_lcc, v=node)
            max_ecc = max(max_ecc, ecc)
        graph_features['diameter'] = max_ecc
        print(f"[4/16] Diameter (approx, 100 samples): {graph_features['diameter']}  ({time.time()-t0:.1f}s)")

# Average path length: expensive, sample-based for large graphs
t1 = time.time()
if n_nodes < 20000:
    graph_features['avg_path_length'] = nx.average_shortest_path_length(G_lcc)
    print(f"        Avg path length: {graph_features['avg_path_length']:.4f}  ({time.time()-t1:.1f}s)")
else:
    # Sample-based estimation
    sample_size = min(500, n_nodes)
    sample_nodes = np.random.choice(list(G_lcc.nodes()), size=sample_size, replace=False)
    total_dist = 0
    count = 0
    for node in sample_nodes:
        lengths = nx.single_source_shortest_path_length(G_lcc, node)
        total_dist += sum(lengths.values())
        count += len(lengths) - 1  # exclude self
    graph_features['avg_path_length'] = total_dist / count if count > 0 else float('inf')
    print(f"        Avg path length (sampled, {sample_size} sources): "
          f"{graph_features['avg_path_length']:.4f}  ({time.time()-t1:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------
# 5. ALGEBRAIC CONNECTIVITY (Fiedler Value)
# --------------------------------------------------------------------------
# Definition: Second smallest eigenvalue of the Laplacian L = D - A:
#   a(G) = μ₂(L),  0 = μ₁ ≤ μ₂ ≤ ... ≤ μ_n
# μ₂ > 0 iff G is connected. Bounds: μ₂ ≤ κ_v(G) ≤ κ_e(G).
# Higher values → harder to partition.
# For large graphs: use scipy.sparse.linalg.eigsh with k=2, which='SM'.
# Citation: Fiedler, M. "Algebraic connectivity of graphs,"
#           Czechoslovak Mathematical Journal 23, 298-305 (1973).
# --------------------------------------------------------------------------

t0 = time.time()

if n_nodes < 30000:
    try:
        graph_features['algebraic_connectivity'] = nx.algebraic_connectivity(
            G_lcc, method='tracemin_pcg'
        )
        print(f"[5/16] Algebraic connectivity: {graph_features['algebraic_connectivity']:.6f}  ({time.time()-t0:.1f}s)")
    except Exception as e:
        print(f"[5/16] Algebraic connectivity: FAILED ({e})")
        graph_features['algebraic_connectivity'] = None
else:
    # Use sparse eigenvalue solver for large graphs
    try:
        L_sparse = nx.laplacian_matrix(G_lcc).astype(float)
        # Compute 2 smallest eigenvalues
        eigenvalues = eigsh(L_sparse, k=2, which='SM', return_eigenvectors=False)
        graph_features['algebraic_connectivity'] = float(np.sort(eigenvalues)[1])
        print(f"[5/16] Algebraic connectivity: {graph_features['algebraic_connectivity']:.6f}  "
              f"({time.time()-t0:.1f}s, sparse eigsh)")
    except Exception as e:
        print(f"[5/16] Algebraic connectivity: FAILED ({e})")
        graph_features['algebraic_connectivity'] = None

In [ ]:
# --------------------------------------------------------------------------
# 6. SPECTRAL RADIUS (Largest Eigenvalue of Adjacency Matrix)
# --------------------------------------------------------------------------
# Definition: λ₁ = max_i |λ_i(A)|
# By Perron-Frobenius: λ₁ is positive and real for connected graphs.
# Bounds: d̄ ≤ λ₁ ≤ d_max. Governs epidemic spreading dynamics.
# Citation: Cvetković, D. et al. "An Introduction to the Theory of Graph
#           Spectra," Cambridge University Press (2010).
# --------------------------------------------------------------------------

t0 = time.time()
try:
    A_sparse = nx.adjacency_matrix(G_lcc).astype(float)
    spectral_radius_vals = eigsh(A_sparse, k=1, which='LM', return_eigenvectors=False)
    graph_features['spectral_radius'] = float(spectral_radius_vals[0])
    print(f"[6/16] Spectral radius: {graph_features['spectral_radius']:.4f}  ({time.time()-t0:.1f}s)")
except Exception as e:
    print(f"[6/16] Spectral radius: FAILED ({e})")
    graph_features['spectral_radius'] = None

In [ ]:
# --------------------------------------------------------------------------
# 7. PERCOLATION LIMIT / EPIDEMIC THRESHOLD
# --------------------------------------------------------------------------
# For the SIS epidemic model: τ_c ≥ 1/λ₁(A)
# Epidemic persists if β/δ > 1/λ₁, dies out below.
# Scale-free AS topologies have vanishing threshold as n → ∞.
# Citation: Pastor-Satorras, R. & Vespignani, A. "Epidemic Spreading in
#           Scale-Free Networks," Phys. Rev. Lett. 86, 3200-3203 (2001).
# --------------------------------------------------------------------------

if graph_features.get('spectral_radius'):
    graph_features['percolation_limit'] = 1.0 / graph_features['spectral_radius']
    print(f"[7/16] Percolation limit: {graph_features['percolation_limit']:.6f}")
    print(f"        → Worms/epidemics can spread at infection rates above {graph_features['percolation_limit']:.4f}")
else:
    graph_features['percolation_limit'] = None
    print(f"[7/16] Percolation limit: SKIPPED (requires spectral radius)")

In [ ]:
# --------------------------------------------------------------------------# 8-11. SPECTRAL METRICS (Conditional on COMPUTE_SPECTRAL flag)# --------------------------------------------------------------------------if COMPUTE_SPECTRAL:    t0 = time.time()    # Compute adjacency & Laplacian matrices ONCE and reuse    L_sparse = nx.laplacian_matrix(G_lcc).astype(float)    A_sparse = nx.adjacency_matrix(G_lcc).astype(float)    n_eigs = min(n_nodes - 2, 300)    use_full_spectrum = n_nodes < 5000    if use_full_spectrum:        print(f"Computing full spectrum ({n_nodes} eigenvalues)...")        L_dense = L_sparse.toarray()        A_dense = A_sparse.toarray()        laplacian_eigs = np.sort(np.real(np.linalg.eigvalsh(L_dense)))        adjacency_eigs = np.sort(np.real(np.linalg.eigvalsh(A_dense)))[::-1]    else:        print(f"Computing partial spectrum ({n_eigs} eigenvalues)...")        laplacian_eigs_small = eigsh(L_sparse, k=min(n_eigs, n_nodes-2),                                     which='SM', return_eigenvectors=False)        laplacian_eigs = np.sort(laplacian_eigs_small)        adjacency_eigs_large = eigsh(A_sparse, k=min(n_eigs, n_nodes-2),                                     which='LM', return_eigenvectors=False)        adjacency_eigs = np.sort(adjacency_eigs_large)[::-1]    print(f"  Spectrum computation: {time.time()-t0:.1f}s")    # ------------------------------------------------------------------    # 8. SYMMETRY RATIO    # SR(G) = |distinct eigenvalues of A| / (D + 1)    # FIX: For partial spectrum, flag as lower bound.    # Citation: Dekker, CATS (2005).    # ------------------------------------------------------------------    distinct_eigs = len(np.unique(np.round(adjacency_eigs, 8)))    D = graph_features.get('diameter', 10)    if use_full_spectrum:        graph_features['symmetry_ratio'] = distinct_eigs / (D + 1)        print(f"[8/16] Symmetry ratio: {graph_features['symmetry_ratio']:.4f} "              f"({distinct_eigs} distinct eigs / {D+1})")    else:        graph_features['symmetry_ratio'] = distinct_eigs / (D + 1)        graph_features['symmetry_ratio_partial'] = True        print(f"[8/16] Symmetry ratio (lower bound): {graph_features['symmetry_ratio']:.4f} "              f"({distinct_eigs} distinct of {len(adjacency_eigs)} computed eigs / {D+1})")        print(f"        NOTE: Partial spectrum - true ratio >= this value")    # ------------------------------------------------------------------    # 9. NATURAL CONNECTIVITY    # lambda_bar = ln[(1/n) sum exp(lambda_i)]    # FIX: For partial spectrum, denominator must be n (total nodes),    # not len(computed eigenvalues), to avoid upward bias.    # Citation: Wu et al., IEEE Trans. SMC-A 41 (2011).    # ------------------------------------------------------------------    if use_full_spectrum:        max_eig = np.max(adjacency_eigs)        graph_features['natural_connectivity'] = float(            max_eig + np.log(np.mean(np.exp(adjacency_eigs - max_eig)))        )        print(f"[9/16] Natural connectivity: {graph_features['natural_connectivity']:.4f}")    else:        max_eig = np.max(adjacency_eigs)        shifted = np.exp(adjacency_eigs - max_eig)        # Use n_nodes as denominator, not len(adjacency_eigs)        graph_features['natural_connectivity'] = float(            max_eig + np.log(np.sum(shifted) / n_nodes)        )        print(f"[9/16] Natural connectivity (approx, {len(adjacency_eigs)} of {n_nodes} eigs, "              f"denominator=n): {graph_features['natural_connectivity']:.4f}")    # ------------------------------------------------------------------    # 10. EFFECTIVE GRAPH RESISTANCE (Kirchhoff Index)    # K_f(G) = n * sum_{k>=2} 1/mu_k    # Citation: Klein & Randic, J. Math. Chem. 12 (1993).    # ------------------------------------------------------------------    nonzero_lap_eigs = laplacian_eigs[laplacian_eigs > 1e-10]    if len(nonzero_lap_eigs) > 0:        graph_features['kirchhoff_index'] = float(n_nodes * np.sum(1.0 / nonzero_lap_eigs))        label = "" if use_full_spectrum else f" (partial, {len(nonzero_lap_eigs)} eigs)"        print(f"[10/16] Kirchhoff index{label}: {graph_features['kirchhoff_index']:.2f}")    else:        graph_features['kirchhoff_index'] = None        print(f"[10/16] Kirchhoff index: SKIPPED (no nonzero Laplacian eigenvalues)")    # ------------------------------------------------------------------    # 11. NUMBER OF SPANNING TREES (requires full spectrum)    # Kirchhoff's Matrix Tree Theorem: tau(G) = (1/n) prod_{k>=2} mu_k    # Citation: Kirchhoff (1847); Godsil & Royle, Springer (2001).    # ------------------------------------------------------------------    if use_full_spectrum and len(nonzero_lap_eigs) > 0:        log_spanning_trees = np.sum(np.log(nonzero_lap_eigs)) - np.log(n_nodes)        graph_features['log_spanning_trees'] = float(log_spanning_trees)        print(f"[11/16] log(spanning trees): {graph_features['log_spanning_trees']:.2f}")    else:        graph_features['log_spanning_trees'] = None        print(f"[11/16] Spanning trees: SKIPPED (requires full spectrum)")else:    print("[8-11] Spectral metrics SKIPPED (COMPUTE_SPECTRAL=False)")    graph_features['symmetry_ratio'] = None    graph_features['natural_connectivity'] = None    graph_features['kirchhoff_index'] = None    graph_features['log_spanning_trees'] = None

In [ ]:
# --------------------------------------------------------------------------
# 12. NODE CONNECTIVITY & EDGE CONNECTIVITY
# --------------------------------------------------------------------------
# Node connectivity κ_v(G): min vertices to remove to disconnect G.
# Edge connectivity κ_e(G): min edges to remove to disconnect G.
# Whitney's inequality: κ_v(G) ≤ κ_e(G) ≤ δ(G).
# By Menger's theorem: equals max disjoint paths between some pair.
# Citation: Whitney, H. "Congruent graphs and the connectivity of graphs,"
#           American J. Math. 54(1), 150-168 (1932).
# --------------------------------------------------------------------------

t0 = time.time()

# Edge connectivity (faster than node connectivity)
try:
    graph_features['edge_connectivity'] = nx.edge_connectivity(G_lcc)
    print(f"[12/16] Edge connectivity: {graph_features['edge_connectivity']}  ({time.time()-t0:.1f}s)")
except Exception as e:
    graph_features['edge_connectivity'] = None
    print(f"[12/16] Edge connectivity: FAILED ({e})")

# Node connectivity
t1 = time.time()
try:
    graph_features['node_connectivity'] = nx.node_connectivity(G_lcc)
    print(f"        Node connectivity: {graph_features['node_connectivity']}  ({time.time()-t1:.1f}s)")
except Exception as e:
    graph_features['node_connectivity'] = None
    print(f"        Node connectivity: FAILED ({e})")

min_degree = min(d for _, d in G_lcc.degree())
print(f"        Min degree δ(G): {min_degree}")
print(f"        Whitney check: κ_v ≤ κ_e ≤ δ → "
      f"{graph_features.get('node_connectivity','?')} ≤ "
      f"{graph_features.get('edge_connectivity','?')} ≤ {min_degree}")

In [ ]:
# --------------------------------------------------------------------------
# 13. RICH-CLUB COEFFICIENT
# --------------------------------------------------------------------------
# Definition: φ(k) = 2E_{>k} / [N_{>k}(N_{>k} - 1)]
#   N_{>k} = nodes with degree > k, E_{>k} = edges among them.
# Normalized: ρ(k) = φ(k) / φ_rand(k). ρ(k) > 1 → rich-club ordering.
# Internet: strong rich-club (Tier-1 providers form dense core).
# Citation: Zhou, S. & Mondragón, R.J. "The Rich-Club Phenomenon in the
#           Internet Topology," IEEE Comm. Lett. 8(3), 180-182 (2004).
#           Colizza, V. et al. "Detecting rich-club ordering in complex
#           networks," Nature Physics 2, 110-115 (2006).
# --------------------------------------------------------------------------

t0 = time.time()
try:
    rc = nx.rich_club_coefficient(G_lcc, normalized=False)
    # Store at selected degree thresholds
    rc_keys = sorted(rc.keys())
    # Sample at percentiles of degree distribution
    p25_k = int(np.percentile(degrees, 25))
    p50_k = int(np.percentile(degrees, 50))
    p75_k = int(np.percentile(degrees, 75))
    p90_k = int(np.percentile(degrees, 90))
    p95_k = int(np.percentile(degrees, 95))
    
    graph_features['rich_club_p25'] = rc.get(p25_k, None)
    graph_features['rich_club_p50'] = rc.get(p50_k, None)
    graph_features['rich_club_p75'] = rc.get(p75_k, None)
    graph_features['rich_club_p90'] = rc.get(p90_k, None)
    graph_features['rich_club_p95'] = rc.get(p95_k, None)
    
    print(f"[13/16] Rich-club coefficient (unnormalized)  ({time.time()-t0:.1f}s)")
    print(f"        φ(k={p25_k}): {graph_features['rich_club_p25']:.6f}" if graph_features['rich_club_p25'] else "")
    print(f"        φ(k={p50_k}): {graph_features['rich_club_p50']:.6f}" if graph_features['rich_club_p50'] else "")
    print(f"        φ(k={p75_k}): {graph_features['rich_club_p75']:.6f}" if graph_features['rich_club_p75'] else "")
    print(f"        φ(k={p90_k}): {graph_features['rich_club_p90']:.6f}" if graph_features['rich_club_p90'] else "")
    print(f"        φ(k={p95_k}): {graph_features['rich_club_p95']:.6f}" if graph_features['rich_club_p95'] else "")
    
    # Store full RC curve for plotting
    rc_df = pd.DataFrame({'k': list(rc.keys()), 'phi': list(rc.values())})
    rc_df.to_csv(OUTPUT_DIR / 'rich_club_curve.csv', index=False)
    
except Exception as e:
    print(f"[13/16] Rich-club coefficient: FAILED ({e})")
    graph_features['rich_club_p50'] = None

In [ ]:
# --------------------------------------------------------------------------# 14. BETWEENNESS CENTRALITY DISTRIBUTION STATISTICS# --------------------------------------------------------------------------# Compute C_B(v) ONCE here, store _bc_map for node-level reuse.# Citation: Brandes, U. J. Math. Sociology 25(2), 163-177 (2001).# --------------------------------------------------------------------------t0 = time.time()if HAS_NETWORKIT:    if BETWEENNESS_SAMPLE_K:        bc_algo = nk.centrality.ApproxBetweenness(G_nk, epsilon=0.01, delta=0.1)        bc_algo.run()        bc_scores_nk = bc_algo.scores()        print(f"[14/16] Betweenness distribution (NetworKit approx)  ({time.time()-t0:.1f}s)")    else:        bc_algo = nk.centrality.Betweenness(G_nk, normalized=True)        bc_algo.run()        bc_scores_nk = bc_algo.scores()        print(f"[14/16] Betweenness distribution (NetworKit exact)  ({time.time()-t0:.1f}s)")    _bc_map = {nk2nx_map[i]: bc_scores_nk[i] for i in range(len(bc_scores_nk))}    bc_scores = np.array(list(_bc_map.values()))else:    _bc_map = nx.betweenness_centrality(G_lcc, k=BETWEENNESS_SAMPLE_K, normalized=True)    bc_scores = np.array(list(_bc_map.values()))    label = f"sampled k={BETWEENNESS_SAMPLE_K}" if BETWEENNESS_SAMPLE_K else "exact"    print(f"[14/16] Betweenness distribution (NetworkX {label})  ({time.time()-t0:.1f}s)")graph_features['betweenness_mean'] = float(np.mean(bc_scores))graph_features['betweenness_max'] = float(np.max(bc_scores))graph_features['betweenness_std'] = float(np.std(bc_scores))graph_features['betweenness_skewness'] = float(sp_stats.skew(bc_scores))print(f"        Mean: {graph_features['betweenness_mean']:.8f}")print(f"        Max:  {graph_features['betweenness_max']:.8f}")print(f"        Std:  {graph_features['betweenness_std']:.8f}")print(f"        Skew: {graph_features['betweenness_skewness']:.4f}")

In [ ]:
# --------------------------------------------------------------------------# 15. K-CORE DECOMPOSITION METRICS# --------------------------------------------------------------------------# Compute ONCE here, store _core_map for node-level reuse.# Citation: Seidman, Social Networks 5(3), 269-287 (1983).# --------------------------------------------------------------------------t0 = time.time()if HAS_NETWORKIT:    cd = nk.centrality.CoreDecomposition(G_nk)    cd.run()    _core_scores_nk = cd.scores()    _core_map = {nk2nx_map[i]: int(_core_scores_nk[i]) for i in range(len(_core_scores_nk))}    core_numbers = np.array(list(_core_map.values()))    graph_features['degeneracy'] = int(cd.maxCoreNumber())else:    _core_map = nx.core_number(G_lcc)    core_numbers = np.array(list(_core_map.values()))    graph_features['degeneracy'] = int(np.max(core_numbers))graph_features['core_mean'] = float(np.mean(core_numbers))graph_features['core_std'] = float(np.std(core_numbers))graph_features['core_median'] = float(np.median(core_numbers))innermost_count = int(np.sum(core_numbers == graph_features['degeneracy']))graph_features['innermost_core_size'] = innermost_countprint(f"[15/16] k-Core decomposition  ({time.time()-t0:.1f}s)")print(f"        Degeneracy (k_max): {graph_features['degeneracy']}")print(f"        Mean core number: {graph_features['core_mean']:.2f}")print(f"        Median core number: {graph_features['core_median']:.1f}")print(f"        Innermost core size: {innermost_count} nodes")

In [ ]:
# --------------------------------------------------------------------------
# 16. WEIGHTED SPECTRUM STATISTICS
# --------------------------------------------------------------------------
# The eigenvalue spectrum of the adjacency (or Laplacian) matrix,
# summarized via: spectral gap (λ₁ - λ₂), spectral norm,
# and normalized Laplacian spectral gap.
# In AS topology, the spectral gap relates to expansion properties
# and mixing time of random walks.
# Citation: Chung, F. "Spectral Graph Theory," AMS (1997).
# --------------------------------------------------------------------------

if COMPUTE_SPECTRAL:
    # Spectral gap = λ₁ - λ₂ of adjacency matrix
    if len(adjacency_eigs) >= 2:
        sorted_adj_eigs = np.sort(adjacency_eigs)[::-1]  # descending
        graph_features['spectral_gap'] = float(sorted_adj_eigs[0] - sorted_adj_eigs[1])
        graph_features['adj_eig_ratio_1_2'] = float(sorted_adj_eigs[0] / sorted_adj_eigs[1]) if sorted_adj_eigs[1] != 0 else None
        print(f"[16/16] Spectral gap (λ₁-λ₂): {graph_features['spectral_gap']:.4f}")
        print(f"        λ₁/λ₂ ratio: {graph_features['adj_eig_ratio_1_2']:.4f}" if graph_features['adj_eig_ratio_1_2'] else "")
    else:
        graph_features['spectral_gap'] = None
        print(f"[16/16] Spectral gap: SKIPPED (insufficient eigenvalues)")
else:
    graph_features['spectral_gap'] = None
    print(f"[16/16] Spectral gap: SKIPPED (COMPUTE_SPECTRAL=False)")

print(f"\n{'='*70}")
print("GRAPH-LEVEL FEATURE EXTRACTION COMPLETE")
print(f"{'='*70}")

## 7. Node-Level Feature Extraction

Extract 10 node-level metrics for every AS in the topology graph.

Each metric includes its mathematical definition, interpretation in AS topology context,
and authoritative citation.

In [ ]:
# ============================================================================
# Node-Level Feature Extraction
# ============================================================================

print(f"Extracting node-level features for {n_nodes:,} nodes...")
print("=" * 70)

# Initialize DataFrame with ASN as index
node_features = pd.DataFrame(index=sorted(G_lcc.nodes()))
node_features.index.name = 'asn'

In [ ]:
# --------------------------------------------------------------------------
# 1. DEGREE CENTRALITY
# --------------------------------------------------------------------------
# Definition: C_D(v) = deg(v) / (n - 1)
# Identifies major transit providers in AS topology.
# Citation: Freeman, L.C. "Centrality in social networks: Conceptual
#           clarification," Social Networks 1(3), 215-239 (1979).
# --------------------------------------------------------------------------

t0 = time.time()
dc = nx.degree_centrality(G_lcc)
node_features['degree_centrality'] = node_features.index.map(dc)
node_features['degree'] = node_features.index.map(dict(G_lcc.degree()))
print(f"[1/10] Degree centrality  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------# 2. BETWEENNESS CENTRALITY (reused from graph-level computation)# --------------------------------------------------------------------------# Already computed above. Reuse _bc_map to avoid duplicate O(nm) work.# Citation: Brandes, U. J. Math. Sociology 25(2), 163-177 (2001).# --------------------------------------------------------------------------t0 = time.time()node_features['betweenness_centrality'] = node_features.index.map(_bc_map)print(f"[2/10] Betweenness centrality (reused from graph-level)  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------
# 3. CLOSENESS CENTRALITY
# --------------------------------------------------------------------------
# Definition: C_C(v) = (n - 1) / Σ_{u≠v} d(v, u)
# Measures how quickly an AS reaches all others.
# Wasserman-Faust variant handles disconnected graphs.
# Citation: Sabidussi, G. "The centrality index of a graph,"
#           Psychometrika 31(4), 581-603 (1966).
# --------------------------------------------------------------------------

t0 = time.time()

if HAS_NETWORKIT:
    cc_algo = nk.centrality.Closeness(G_nk, True, nk.centrality.ClosenessVariant.GENERALIZED)
    cc_algo.run()
    cc_scores = cc_algo.scores()
    cc_map = {nk2nx_map[i]: cc_scores[i] for i in range(len(cc_scores))}
    node_features['closeness_centrality'] = node_features.index.map(cc_map)
    print(f"[3/10] Closeness centrality (NetworKit)  ({time.time()-t0:.1f}s)")
else:
    cc = nx.closeness_centrality(G_lcc, wf_improved=True)
    node_features['closeness_centrality'] = node_features.index.map(cc)
    print(f"[3/10] Closeness centrality (NetworkX)  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------
# 4. EIGENVECTOR CENTRALITY
# --------------------------------------------------------------------------
# Definition: Principal eigenvector of A: Ax = λ₁x.
# A node's centrality ∝ sum of neighbors' centralities.
# Citation: Bonacich, P. "Factoring and weighting approaches to status
#           scores and clique identification," J. Math. Sociology 2(1),
#           113-120 (1972).
# --------------------------------------------------------------------------

t0 = time.time()

if HAS_NETWORKIT:
    ev_algo = nk.centrality.EigenvectorCentrality(G_nk, tol=1e-8)
    ev_algo.run()
    ev_scores = ev_algo.scores()
    ev_map = {nk2nx_map[i]: ev_scores[i] for i in range(len(ev_scores))}
    node_features['eigenvector_centrality'] = node_features.index.map(ev_map)
    print(f"[4/10] Eigenvector centrality (NetworKit)  ({time.time()-t0:.1f}s)")
else:
    try:
        ev = nx.eigenvector_centrality(G_lcc, max_iter=200, tol=1e-6)
        node_features['eigenvector_centrality'] = node_features.index.map(ev)
        print(f"[4/10] Eigenvector centrality (NetworkX)  ({time.time()-t0:.1f}s)")
    except nx.PowerIterationFailedConvergence:
        ev = nx.eigenvector_centrality_numpy(G_lcc)
        node_features['eigenvector_centrality'] = node_features.index.map(ev)
        print(f"[4/10] Eigenvector centrality (NetworkX numpy fallback)  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------
# 5. PAGERANK
# --------------------------------------------------------------------------
# Definition: Stationary distribution of random walk with damping d=0.85:
#   PR(v) = (1-d)/n + d Σ_{u∈N(v)} PR(u)/deg(u)
# Citation: Brin, S. & Page, L. "The anatomy of a large-scale hypertextual
#           web search engine," Computer Networks 30(1-7), 107-117 (1998).
# --------------------------------------------------------------------------

t0 = time.time()

if HAS_NETWORKIT:
    pr_algo = nk.centrality.PageRank(G_nk, damp=0.85, tol=1e-8)
    pr_algo.run()
    pr_scores = pr_algo.scores()
    pr_map = {nk2nx_map[i]: pr_scores[i] for i in range(len(pr_scores))}
    node_features['pagerank'] = node_features.index.map(pr_map)
    print(f"[5/10] PageRank (NetworKit, d=0.85)  ({time.time()-t0:.1f}s)")
else:
    pr = nx.pagerank(G_lcc, alpha=0.85)
    node_features['pagerank'] = node_features.index.map(pr)
    print(f"[5/10] PageRank (NetworkX, d=0.85)  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------
# 6. LOCAL CLUSTERING COEFFICIENT
# --------------------------------------------------------------------------
# Definition: C(v) = 2·triangles(v) / [d_v(d_v - 1)]
# High clustering at stub ASes → regional peering clusters.
# Low clustering at transit ASes.
# Citation: Watts & Strogatz, Nature 393 (1998).
# --------------------------------------------------------------------------

t0 = time.time()

if HAS_NETWORKIT:
    lcc_algo = nk.centrality.LocalClusteringCoefficient(G_nk, turbo=True)
    lcc_algo.run()
    lcc_scores = lcc_algo.scores()
    lcc_map = {nk2nx_map[i]: lcc_scores[i] for i in range(len(lcc_scores))}
    node_features['local_clustering'] = node_features.index.map(lcc_map)
    print(f"[6/10] Local clustering coefficient (NetworKit turbo)  ({time.time()-t0:.1f}s)")
else:
    clust = nx.clustering(G_lcc)
    node_features['local_clustering'] = node_features.index.map(clust)
    print(f"[6/10] Local clustering coefficient (NetworkX)  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------
# 7. AVERAGE NEIGHBOR DEGREE
# --------------------------------------------------------------------------
# Definition: k_nn(v) = (1/d_v) Σ_{u∈N(v)} d_u
# Decreasing k_nn(v) vs d_v confirms disassortative mixing in AS topology.
# Citation: Pastor-Satorras, R. et al. "Dynamical and correlation properties
#           of the Internet," Phys. Rev. Lett. 87(25), 258701 (2001).
# --------------------------------------------------------------------------

t0 = time.time()
and_dict = nx.average_neighbor_degree(G_lcc)
node_features['avg_neighbor_degree'] = node_features.index.map(and_dict)
print(f"[7/10] Average neighbor degree  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------# 8. NODE CLIQUE NUMBER# --------------------------------------------------------------------------# NP-hard. Exact for small graphs, greedy approximation for large ones.# Citation: Standard NP-completeness result (Karp, 1972).# --------------------------------------------------------------------------t0 = time.time()if n_nodes <= MAX_NODES_FOR_CLIQUE:    ncn = nx.node_clique_number(G_lcc)    node_features['node_clique_number'] = node_features.index.map(ncn)    print(f"[8/10] Node clique number (exact)  ({time.time()-t0:.1f}s)")else:    # Greedy approximation: expand from each node by adding highest-degree    # neighbors that form a clique    def greedy_clique_size(G, node):        clique = {node}        candidates = set(G.neighbors(node))        for cand in sorted(candidates, key=lambda x: G.degree(x), reverse=True):            if all(G.has_edge(cand, c) for c in clique):                clique.add(cand)        return len(clique)    k_max = graph_features['degeneracy']    core_subgraph = nx.k_core(G_lcc, k=k_max)    ncn_approx = {}    # Exact for innermost core if small enough    if core_subgraph.number_of_nodes() <= MAX_NODES_FOR_CLIQUE:        ncn_core = nx.node_clique_number(core_subgraph)        ncn_approx.update(ncn_core)        print(f"  Exact clique number for {core_subgraph.number_of_nodes()} innermost core nodes")    # Greedy for remaining    for node in G_lcc.nodes():        if node not in ncn_approx:            ncn_approx[node] = greedy_clique_size(G_lcc, node)    node_features['node_clique_number'] = node_features.index.map(ncn_approx)    print(f"[8/10] Node clique number (core exact + greedy approx)  ({time.time()-t0:.1f}s)")

In [ ]:
# --------------------------------------------------------------------------# 9. ECCENTRICITY# --------------------------------------------------------------------------# FIX: For large graphs use NetworKit BFS (much faster than NetworkX),# fall back to sampling only when NetworKit is unavailable.# Citation: Standard graph theory definition.# --------------------------------------------------------------------------t0 = time.time()if n_nodes < 30000:    ecc = nx.eccentricity(G_lcc)    node_features['eccentricity'] = node_features.index.map(ecc)    graph_features['radius'] = min(ecc.values())    print(f"[9/10] Eccentricity  ({time.time()-t0:.1f}s)")    print(f"        Radius: {graph_features['radius']}")elif HAS_NETWORKIT:    # NetworKit BFS for all nodes — much faster than NetworkX    ecc_map = {}    node_list = sorted(G_lcc.nodes())    nx_to_nk_local = {n: i for i, n in enumerate(node_list)}    for node in node_list:        bfs = nk.distance.BFS(G_nk, nx_to_nk_local[node])        bfs.run()        distances = bfs.getDistances()        reachable = [d for d in distances if d < float('inf')]        ecc_map[node] = int(max(reachable)) if reachable else 0    node_features['eccentricity'] = node_features.index.map(ecc_map)    graph_features['radius'] = min(ecc_map.values())    print(f"[9/10] Eccentricity (NetworKit BFS, all nodes)  ({time.time()-t0:.1f}s)")    print(f"        Radius: {graph_features['radius']}")else:    # Sample-based for large graphs without NetworKit    sample_size = min(500, n_nodes)    sample_nodes = np.random.choice(list(G_lcc.nodes()), size=sample_size, replace=False)    ecc_sample = {}    for node in sample_nodes:        lengths = nx.single_source_shortest_path_length(G_lcc, node)        ecc_sample[node] = max(lengths.values())    node_features['eccentricity'] = node_features.index.map(        lambda x: ecc_sample.get(x, np.nan)    )    graph_features['radius'] = min(ecc_sample.values()) if ecc_sample else None    print(f"[9/10] Eccentricity (sampled, {sample_size} nodes)  ({time.time()-t0:.1f}s)")    print(f"        Approx radius: {graph_features.get('radius')}")

In [ ]:
# --------------------------------------------------------------------------# 10. K-SHELL / CORE NUMBER (reused from graph-level computation)# --------------------------------------------------------------------------# Reuse _core_map computed above. No duplicate computation.# Citation: Seidman, Social Networks 5(3), 269-287 (1983).# --------------------------------------------------------------------------t0 = time.time()node_features['core_number'] = node_features.index.map(_core_map)print(f"[10/10] Core number (reused from graph-level)  ({time.time()-t0:.1f}s)")print(f"\n{'='*70}")print("NODE-LEVEL FEATURE EXTRACTION COMPLETE")print(f"{'='*70}")print(f"\nNode features shape: {node_features.shape}")node_features.describe()

## 8. Results Summary & Export

In [ ]:
# ============================================================================
# Summary of Graph-Level Features
# ============================================================================

print("\n" + "=" * 70)
print("GRAPH-LEVEL FEATURES SUMMARY")
print("=" * 70)
print(f"{'Feature':<35} {'Value':>20} {'Citation'}")
print("-" * 90)

feature_citations = {
    'n_nodes': 'Graph property',
    'n_edges': 'Graph property',
    'assortativity': 'Newman, Phys.Rev.Lett. 89 (2002)',
    'density': 'Standard',
    'clustering_global': 'Watts & Strogatz, Nature 393 (1998)',
    'clustering_avg_local': 'Watts & Strogatz, Nature 393 (1998)',
    'diameter': 'Watts & Strogatz, Nature 393 (1998)',
    'avg_path_length': 'Watts & Strogatz, Nature 393 (1998)',
    'algebraic_connectivity': 'Fiedler, Czech.Math.J. 23 (1973)',
    'spectral_radius': 'Cvetković et al., Cambridge (2010)',
    'percolation_limit': 'Pastor-Satorras, Phys.Rev.Lett. 86 (2001)',
    'symmetry_ratio': 'Dekker, CATS (2005)',
    'natural_connectivity': 'Wu et al., IEEE Trans. SMC-A 41 (2011)',
    'kirchhoff_index': 'Klein & Randić, J.Math.Chem. 12 (1993)',
    'log_spanning_trees': 'Kirchhoff (1847)',
    'edge_connectivity': 'Whitney, Am.J.Math. 54 (1932)',
    'node_connectivity': 'Whitney, Am.J.Math. 54 (1932)',
    'rich_club_p50': 'Zhou & Mondragón, IEEE Comm.Lett. (2004)',
    'rich_club_p90': 'Zhou & Mondragón, IEEE Comm.Lett. (2004)',
    'betweenness_mean': 'Brandes, J.Math.Soc. 25 (2001)',
    'betweenness_max': 'Brandes, J.Math.Soc. 25 (2001)',
    'betweenness_std': 'Brandes, J.Math.Soc. 25 (2001)',
    'betweenness_skewness': 'Brandes, J.Math.Soc. 25 (2001)',
    'degeneracy': 'Seidman, Social Networks 5 (1983)',
    'core_mean': 'Seidman, Social Networks 5 (1983)',
    'innermost_core_size': 'Alvarez-Hamelin et al. (2008)',
    'spectral_gap': 'Chung, AMS (1997)',
    'radius': 'Standard',
}

for feat, val in graph_features.items():
    if val is not None:
        if isinstance(val, float):
            val_str = f"{val:.6f}" if abs(val) < 100 else f"{val:.2f}"
        else:
            val_str = f"{val:,}" if isinstance(val, int) else str(val)
        citation = feature_citations.get(feat, '')
        print(f"{feat:<35} {val_str:>20}  {citation}")

In [ ]:
# ============================================================================
# Export Results
# ============================================================================

# 1. Graph-level features as JSON
graph_features_serializable = {}
for k, v in graph_features.items():
    if isinstance(v, (np.integer, np.int64)):
        graph_features_serializable[k] = int(v)
    elif isinstance(v, (np.floating, np.float64)):
        graph_features_serializable[k] = float(v)
    else:
        graph_features_serializable[k] = v

with open(OUTPUT_DIR / 'graph_level_features.json', 'w') as f:
    json.dump(graph_features_serializable, f, indent=2, default=str)

# 2. Node-level features as CSV
node_features.to_csv(OUTPUT_DIR / 'node_level_features.csv')

# 3. Edge list
edges_df = pd.DataFrame(list(G_lcc.edges()), columns=['source_asn', 'target_asn'])
edges_df.to_csv(OUTPUT_DIR / 'as_edges.csv', index=False)

# 4. Graph in multiple formats
nx.write_graphml(G_lcc, str(OUTPUT_DIR / 'as_topology.graphml'))

print(f"Files saved to {OUTPUT_DIR}:")
for f in sorted(OUTPUT_DIR.iterdir()):
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"  {f.name:<35} {size_mb:.2f} MB")

## 9. Visualization

In [ ]:
# ============================================================================
# Visualization: Centrality Correlations
# ============================================================================

centrality_cols = ['degree_centrality', 'betweenness_centrality', 'closeness_centrality',
                   'eigenvector_centrality', 'pagerank']
existing_cols = [c for c in centrality_cols if c in node_features.columns]

if len(existing_cols) >= 2:
    fig, ax = plt.subplots(figsize=(8, 6))
    corr = node_features[existing_cols].corr(method='spearman')
    im = ax.imshow(corr, cmap='RdBu_r', vmin=-1, vmax=1)
    ax.set_xticks(range(len(existing_cols)))
    ax.set_yticks(range(len(existing_cols)))
    short_names = [c.replace('_centrality', '').replace('_', ' ').title() for c in existing_cols]
    ax.set_xticklabels(short_names, rotation=45, ha='right')
    ax.set_yticklabels(short_names)
    
    # Add correlation values
    for i in range(len(existing_cols)):
        for j in range(len(existing_cols)):
            ax.text(j, i, f"{corr.iloc[i, j]:.2f}", ha='center', va='center',
                    color='white' if abs(corr.iloc[i, j]) > 0.5 else 'black', fontsize=10)
    
    plt.colorbar(im, label='Spearman ρ')
    ax.set_title('Centrality Measure Correlations (Spearman)')
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'centrality_correlations.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {FIGURES_DIR / 'centrality_correlations.png'}")

In [ ]:
# ============================================================================
# Visualization: Core Number Distribution
# ============================================================================

if 'core_number' in node_features.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Core number histogram
    core_vals = node_features['core_number'].dropna()
    axes[0].hist(core_vals, bins=range(int(core_vals.min()), int(core_vals.max()) + 2),
                 edgecolor='black', alpha=0.7, color='coral')
    axes[0].set_xlabel('Core Number (k-shell)')
    axes[0].set_ylabel('Number of ASes')
    axes[0].set_title('k-Core Decomposition Distribution')
    axes[0].set_yscale('log')
    
    # Degree vs Core Number scatter
    sample_idx = np.random.choice(len(node_features), size=min(5000, len(node_features)), replace=False)
    sample_df = node_features.iloc[sample_idx]
    axes[1].scatter(sample_df['degree'], sample_df['core_number'],
                    alpha=0.3, s=5, c='steelblue')
    axes[1].set_xlabel('Degree')
    axes[1].set_ylabel('Core Number')
    axes[1].set_title('Degree vs Core Number')
    axes[1].set_xscale('log')
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'kcore_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {FIGURES_DIR / 'kcore_distribution.png'}")

In [ ]:
# ============================================================================
# Visualization: Disassortative Mixing (k_nn vs k)
# ============================================================================

if 'avg_neighbor_degree' in node_features.columns:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Bin by degree for cleaner visualization
    df_plot = node_features[['degree', 'avg_neighbor_degree']].dropna()
    degree_bins = pd.cut(df_plot['degree'], bins=50)
    grouped = df_plot.groupby(degree_bins, observed=True).agg(
        mean_k=('degree', 'mean'),
        mean_knn=('avg_neighbor_degree', 'mean'),
        count=('degree', 'count')
    ).dropna()
    
    ax.scatter(grouped['mean_k'], grouped['mean_knn'],
               s=np.clip(grouped['count'], 1, 500), alpha=0.6, c='steelblue')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Degree k')
    ax.set_ylabel('Average Neighbor Degree k_nn(k)')
    ax.set_title(f'Disassortative Mixing (r = {graph_features["assortativity"]:.4f})')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'disassortative_mixing.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {FIGURES_DIR / 'disassortative_mixing.png'}")

In [ ]:
# ============================================================================
# Visualization: Rich-Club Coefficient
# ============================================================================

rc_path = OUTPUT_DIR / 'rich_club_curve.csv'
if rc_path.exists():
    rc_df = pd.read_csv(rc_path)
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(rc_df['k'], rc_df['phi'], '-', color='crimson', linewidth=1.5)
    ax.set_xlabel('Degree k')
    ax.set_ylabel('φ(k)')
    ax.set_title('Rich-Club Coefficient (Unnormalized)')
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'rich_club_coefficient.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Saved: {FIGURES_DIR / 'rich_club_coefficient.png'}")

In [ ]:
# ============================================================================
# Top ASes by Various Centrality Measures
# ============================================================================

print("\nTop 15 ASes by different centrality measures:")
print("=" * 90)

for metric in ['degree_centrality', 'betweenness_centrality', 'pagerank', 'eigenvector_centrality']:
    if metric in node_features.columns:
        top = node_features[metric].nlargest(15)
        print(f"\n--- {metric.replace('_', ' ').title()} ---")
        for asn, val in top.items():
            core_n = node_features.loc[asn, 'core_number'] if 'core_number' in node_features.columns else '?'
            deg = node_features.loc[asn, 'degree'] if 'degree' in node_features.columns else '?'
            print(f"  AS{asn:<8}  {metric}: {val:.8f}  degree: {deg}  core: {core_n}")

## 10. Feature Definitions Reference

### Graph-Level Features (16)

| # | Feature | Definition | Citation |
|---|---------|------------|----------|
| 1 | **Assortativity** | Pearson correlation of degrees at edge endpoints | Newman, *Phys. Rev. Lett.* 89, 208701 (2002) |
| 2 | **Density** | ρ = 2\|E\| / [\|V\|(\|V\|-1)] | Standard |
| 3 | **Clustering (global)** | 3×triangles / connected triples | Watts & Strogatz, *Nature* 393 (1998) |
| 4 | **Diameter** | max\_{u,v} d(u,v) | Watts & Strogatz (1998) |
| 5 | **Algebraic connectivity** | μ₂(L), 2nd smallest Laplacian eigenvalue | Fiedler, *Czech. Math. J.* 23 (1973) |
| 6 | **Spectral radius** | λ₁(A), largest adjacency eigenvalue | Cvetković et al., Cambridge (2010) |
| 7 | **Percolation limit** | τ\_c = 1/λ₁(A), epidemic threshold | Pastor-Satorras & Vespignani, *PRL* 86 (2001) |
| 8 | **Symmetry ratio** | \|distinct eigenvalues\| / (D+1) | Dekker, CATS (2005) |
| 9 | **Natural connectivity** | ln[(1/n) Σ exp(λᵢ)] | Wu et al., *IEEE Trans. SMC-A* 41 (2011) |
| 10 | **Kirchhoff index** | n Σ\_{k≥2} 1/μ\_k | Klein & Randić, *J. Math. Chem.* 12 (1993) |
| 11 | **log(Spanning trees)** | (1/n) Π\_{k≥2} μ\_k (via matrix tree theorem) | Kirchhoff (1847) |
| 12 | **Edge/node connectivity** | Min cut size | Whitney, *Am. J. Math.* 54 (1932) |
| 13 | **Rich-club coefficient** | φ(k) = 2E\_{>k} / [N\_{>k}(N\_{>k}-1)] | Zhou & Mondragón, *IEEE Comm. Lett.* (2004) |
| 14 | **Betweenness distribution** | Mean, max, std, skewness of C\_B(v) | Brandes, *J. Math. Soc.* 25 (2001) |
| 15 | **k-Core metrics** | Degeneracy k\_max, core distribution | Seidman, *Social Networks* 5 (1983) |
| 16 | **Spectral gap** | λ₁ - λ₂ of adjacency matrix | Chung, *Spectral Graph Theory*, AMS (1997) |

### Node-Level Features (10)

| # | Feature | Definition | Citation |
|---|---------|------------|----------|
| 1 | **Degree centrality** | C\_D(v) = deg(v)/(n-1) | Freeman, *Social Networks* 1 (1979) |
| 2 | **Betweenness centrality** | C\_B(v) = Σ σ\_st(v)/σ\_st | Brandes, *J. Math. Soc.* 25 (2001) |
| 3 | **Closeness centrality** | C\_C(v) = (n-1)/Σ d(v,u) | Sabidussi, *Psychometrika* 31 (1966) |
| 4 | **Eigenvector centrality** | Principal eigenvector of A | Bonacich, *J. Math. Soc.* 2 (1972) |
| 5 | **PageRank** | Stationary random walk dist. (d=0.85) | Brin & Page, *Computer Networks* 30 (1998) |
| 6 | **Local clustering** | C(v) = 2·tri(v)/[d(d-1)] | Watts & Strogatz (1998) |
| 7 | **Avg neighbor degree** | k\_nn(v) = (1/d) Σ\_{u∈N(v)} d\_u | Pastor-Satorras et al., *PRL* 87 (2001) |
| 8 | **Node clique number** | ω(v) = max clique containing v | NP-hard (Karp, 1972) |
| 9 | **Eccentricity** | ε(v) = max\_u d(v,u) | Standard |
| 10 | **Core number (k-shell)** | max{k : v ∈ H\_k} | Seidman, *Social Networks* 5 (1983) |

In [ ]:
print("\n" + "=" * 70)
print("PIPELINE COMPLETE")
print("=" * 70)
print(f"\nCollector: {COLLECTOR}")
print(f"Date range: {START_DATE} to {END_DATE}")
print(f"Graph: {G_lcc.number_of_nodes():,} nodes, {G_lcc.number_of_edges():,} edges")
print(f"Graph-level features: {len([v for v in graph_features.values() if v is not None])}")
print(f"Node-level features: {node_features.shape[1]} columns x {node_features.shape[0]:,} ASes")
print(f"\nOutput directory: {OUTPUT_DIR}")
print(f"Figures directory: {FIGURES_DIR}")
print(f"MRT files directory: {MRT_DIR}")